<a href="https://colab.research.google.com/github/LUCY-JungHwaKim/DataAnalysis/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0%EC%8B%A4%EC%8A%B51111_%EC%A0%9C2%EC%9C%A0%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 펭귄의 무게를 예측하는 회귀 분석

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
# 결과에 큰 영향이 없는 경고 문구들 무시

In [3]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 485 (delta 66), reused 49 (delta 44), pack-reused 363
Receiving objects: 100% (485/485), 8.89 MiB | 17.53 MiB/s, done.
Resolving deltas: 100% (251/251), done.


In [4]:
X_test = pd.read_csv('./yemoonsaBigdata/datasets/Part2/penguin_X_test.csv')
X_train = pd.read_csv('./yemoonsaBigdata/datasets/Part2/penguin_X_train.csv')
y_train = pd.read_csv('./yemoonsaBigdata/datasets/Part2/penguin_y_train.csv')

## 데이터 전처리

In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            240 non-null    object 
 1   island             240 non-null    object 
 2   sex                232 non-null    object 
 3   bill_length_mm     238 non-null    float64
 4   bill_depth_mm      238 non-null    float64
 5   flipper_length_mm  238 non-null    float64
dtypes: float64(3), object(3)
memory usage: 11.4+ KB


In [6]:
train=pd.concat([X_train, y_train], axis=1)

train[train.sex.isna()] ## 결측치 데이터 확인

,species,island,sex,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,Torgersen,NaN,42.0,20.2,190.0,4250.0
6,Gentoo,Biscoe,NaN,44.5,14.3,216.0,4100.0
43,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
66,Adelie,Torgersen,NaN,37.8,17.3,180.0,3700.0
88,Gentoo,Biscoe,NaN,47.3,13.8,216.0,4725.0
89,Adelie,Torgersen,NaN,37.8,17.1,186.0,3300.0
110,Gentoo,Biscoe,NaN,44.5,15.7,217.0,4875.0
229,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN


In [7]:
## 결측치 제거 후 인덱스 
# 8개의 행에서만 데이터 결측이 발생하는 것 같으므로 제거
train = train.dropna()
train.reset_index(drop=True, inplace=True)

In [8]:
X_train = train[['species', 'island', 'sex', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']]
y_train = train[['body_mass_g']]

In [9]:
X_train.describe() ## 스케일링 필요해 보임

,bill_length_mm,bill_depth_mm,flipper_length_mm
count,232.000000,232.000000,232.000000
mean,43.990948,17.226293,200.681034
std,5.509760,1.964677,14.064231
min,32.100000,13.200000,172.000000
25%,39.200000,15.700000,190.000000
50%,44.950000,17.350000,197.000000
75%,48.775000,18.725000,212.250000
max,58.000000,21.500000,231.000000


In [10]:
COL_DEL = 0
## 수치형 변수 --> 스케일링 진행
## 범주형 변수 -> 레이블링 인코딩 필요
COL_NUM = ['bill_depth_mm', 'bill_length_mm', 'flipper_length_mm']
COL_CAT = ['species', 'island', 'sex']
COL_Y = ['body_mass_g']

In [11]:
X = pd.concat([X_train, X_test])

from sklearn.preprocessing import OneHotEncoder
# 범주형 변수 스케일링

ohe = OneHotEncoder(handle_unknown='ignore')
# 처음보는 범주가 등장했을 때 어떻게 처리할 것인지 결정하는 옵션

ohe.fit(X[COL_CAT])

X_train_res = ohe.transform(X_train[COL_CAT])
X_test_res = ohe.transform(X_test[COL_CAT])

In [ ]:
print(X_train_res)

In [15]:
## 데이터프레임으로 변환
X_train_ohe = pd.DataFrame(X_train_res.todense(), columns = ohe.get_feature_names())
X_test_ohe = pd.DataFrame(X_test_res.todense(), columns = ohe.get_feature_names())


In [16]:
X_train_fin = pd.concat([X_train[COL_NUM], X_train_ohe], axis=1)
X_test_fin = pd.concat([X_test[COL_NUM], X_test_ohe], axis=1)

In [17]:
X_train_ohe.head()

,x0_Adelie,x0_Chinstrap,x0_Gentoo,x1_Biscoe,x1_Dream,x1_Torgersen,x2_FEMALE,x2_MALE
0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [19]:
## 데이터 모형 구축
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train_fin, y_train, test_size=0.3)

In [20]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test_fin[COL_NUM] = scaler.transform(X_test_fin[COL_NUM])

In [22]:
## 모형 학습
from sklearn.linear_model import LinearRegression

modelLR = LinearRegression()
modelLR.fit(X_tr, y_tr)

y_val_pred = modelLR.predict(X_val)
y_val_pred


array([[4683.65768791],
       [4614.88514771],
       [3404.14166573],
       [3623.52068661],
       [3579.67197194],
       [4255.53898069],
       [4666.42542849],
       [3391.59491947],
       [5251.77729625],
       [3457.84340031],
       [4018.08105033],
       [4098.58038901],
       [4136.98680573],
       [4284.7089909 ],
       [4103.57114759],
       [3472.7023992 ],
       [5485.82405309],
       [4797.30994708],
       [3213.63761162],
       [3933.45255421],
       [4180.81007205],
       [4727.58303038],
       [4027.60474165],
       [3527.65701212],
       [4659.90782057],
       [5507.56117503],
       [4045.41880515],
       [5374.37490107],
       [4270.38575716],
       [5700.42508074],
       [4051.33936261],
       [4095.6662326 ],
       [5508.54288568],
       [4047.46216988],
       [4016.57031811],
       [3910.19360135],
       [5437.96024068],
       [3497.75364503],
       [5585.0170593 ],
       [3958.36947552],
       [4124.45788542],
       [5483.346

In [23]:
coef = pd.Series(data = modelLR.coef_[0], index=X_train_fin.columns)
coef.sort_values

<bound method Series.sort_values of bill_depth_mm        435.936173
bill_length_mm       459.138052
flipper_length_mm    944.231766
x0_Adelie           -254.631408
x0_Chinstrap        -423.962406
x0_Gentoo            678.593814
x1_Biscoe             56.186619
x1_Dream             -38.466432
x1_Torgersen         -17.720187
x2_FEMALE           -194.151436
x2_MALE              194.151436
dtype: float64>

In [24]:
## 암컷이 수컷보다 작게 나오는 것으로 보임
## Gentoo 종일 때 더 무거움

In [25]:
## 모형 평가
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_val, y_val_pred)
rmse = mean_squared_error(y_val, y_val_pred, squared=False)

mse, rmse

(84455.79550507486, 290.6127930857051)

In [26]:
y_pred = modelLR.predict(X_test_fin)

In [27]:
y_pred ## 요 값이 결과물

array([[4071.62543905],
       [5315.71133254],
       [4070.58577493],
       [3433.61811785],
       [4685.02044782],
       [4000.65393939],
       [3357.16337698],
       [3736.08255006],
       [3551.48353771],
       [3487.90248046],
       [3377.70899376],
       [5472.72756003],
       [3606.3870754 ],
       [4053.55269772],
       [3772.59162121],
       [4087.30093543],
       [4354.6042794 ],
       [3516.49660622],
       [3601.20763344],
       [3347.16500662],
       [5318.67845445],
       [4130.66662889],
       [5834.69012307],
       [4693.13205513],
       [5364.42585482],
       [5485.39869816],
       [5488.50183928],
       [4158.32604154],
       [3434.61753611],
       [4002.27870086],
       [4135.03301097],
       [3409.02949374],
       [3555.63468512],
       [4632.48148485],
       [3350.88833787],
       [3455.36806362],
       [3297.67123131],
       [3375.80612992],
       [3898.35693764],
       [4962.43778189],
       [3484.40131855],
       [5576.742